In [26]:
import pandas as pd
import numpy as np

In [2]:
machines_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PdM_machines.csv")
maint_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PdM_maint.csv")
errors_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PdM_errors.csv")
failures_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PdM_failures.csv")
telemetry_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PdM_telemetry.csv")

In [3]:
telemetry_df['datetime'] = pd.to_datetime(telemetry_df['datetime'])
maint_df['datetime'] = pd.to_datetime(maint_df['datetime'])
failures_df['datetime'] = pd.to_datetime(failures_df['datetime'])
errors_df['datetime'] = pd.to_datetime(errors_df['datetime'])

In [4]:
df = pd.merge(telemetry_df, machines_df, on=['machineID'], how='left')

In [5]:
sorted_df = df.sort_values(by=['machineID', 'datetime'])
merge_df = pd.merge(sorted_df, maint_df[['machineID', 'datetime']], on=['machineID', 'datetime'], how='left')

In [6]:
merge_df

,datetime,machineID,volt,rotate,pressure,vibration,model,age
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18
...,...,...,...,...,...,...,...,...
876818,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5
876819,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5
876820,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5
876821,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5


In [7]:
type(merge_df['datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
df = pd.merge(merge_df, maint_df, on=['machineID', 'datetime'], how='left')

# The key step: sort and forward-fill to get the last maintenance date
df.sort_values(by=['machineID', 'datetime'], inplace=True)

In [9]:
from datetime import timedelta

In [10]:
def compute_C(group):
    # Forward fill B where A is not null
    last_valid_B = group['datetime'].where(group['comp'].notnull()).ffill()
    # Apply the condition
    group['last_maint_datetime'] = np.where(group['comp'].notnull(), group['datetime'] - last_valid_B, (group['datetime'] - last_valid_B))
    return group

In [11]:
df = df.groupby('machineID', group_keys=False).apply(compute_C)

/tmp/ipython-input-3770084564.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('machineID', group_keys=False).apply(compute_C)


In [12]:
df.drop(columns=['comp'], inplace=True)

In [34]:
# First, add a simple count column to the errors DataFrame
errors_df['error_count'] = 1

# Group by machine and time, then use rolling() to get the count
error_counts = errors_df.groupby(['machineID', 'datetime']).count().reset_index()


In [52]:
df.drop(columns=['errorID', 'error_count', 'error_count_in_last_7_days'], inplace=True)

In [59]:
df

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,NaT
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,NaT
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,NaT
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,NaT
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,NaT
...,...,...,...,...,...,...,...,...,...
878567,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5,7 days 20:00:00
878568,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5,7 days 21:00:00
878569,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5,7 days 22:00:00
878570,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5,7 days 23:00:00


In [58]:
error_counts

,machineID,datetime,errorID,error_count
0,1,2015-01-03 07:00:00,1,1
1,1,2015-01-03 20:00:00,1,1
2,1,2015-01-04 06:00:00,1,1
3,1,2015-01-10 15:00:00,1,1
4,1,2015-01-22 10:00:00,1,1
...,...,...,...,...
3611,100,2015-11-12 01:00:00,1,1
3612,100,2015-11-21 08:00:00,1,1
3613,100,2015-12-04 02:00:00,1,1
3614,100,2015-12-08 06:00:00,2,2


In [60]:
df = pd.merge(df, error_counts, on=['machineID', 'datetime'], how='left')

In [62]:
df

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime,errorID,error_count
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,NaT,NaN,NaN
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,NaT,NaN,NaN
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,NaT,NaN,NaN
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,NaT,NaN,NaN
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
878567,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5,7 days 20:00:00,NaN,NaN
878568,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5,7 days 21:00:00,NaN,NaN
878569,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5,7 days 22:00:00,NaN,NaN
878570,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5,7 days 23:00:00,NaN,NaN


In [63]:
df['error_count'] = df['error_count'].fillna(0)

In [72]:
df = df.reset_index()

In [76]:
df['datetime']

,datetime
0,2015-01-01 06:00:00
1,2015-01-01 07:00:00
2,2015-01-01 08:00:00
3,2015-01-01 09:00:00
4,2015-01-01 10:00:00
...,...
878567,2016-01-01 02:00:00
878568,2016-01-01 03:00:00
878569,2016-01-01 04:00:00
878570,2016-01-01 05:00:00


In [75]:
type(df['datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [78]:
df = df.set_index('datetime')
df['error_last_7d'] = df.groupby(['machineID'])['error_count'].rolling(window='7d').sum().reset_index(level=0, drop=True)
df = df.reset_index()

In [84]:
df.drop(columns=['errorID', 'error_count'], inplace=True)

In [85]:
df[210: 240]

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime,error_last_7d
210,2015-01-09 21:00:00,1,194.215872,464.734076,87.693438,42.764628,model3,18,4 days 15:00:00,3.0
211,2015-01-09 22:00:00,1,163.295603,462.106471,87.714024,32.340607,model3,18,4 days 16:00:00,3.0
212,2015-01-09 23:00:00,1,181.447651,552.372925,101.871854,45.133304,model3,18,4 days 17:00:00,3.0
213,2015-01-10 00:00:00,1,192.075780,457.791608,102.703776,40.895894,model3,18,4 days 18:00:00,3.0
214,2015-01-10 01:00:00,1,176.654974,416.429100,108.093708,45.834735,model3,18,4 days 19:00:00,3.0
215,2015-01-10 02:00:00,1,163.674172,418.424650,99.038038,39.024704,model3,18,4 days 20:00:00,3.0
216,2015-01-10 03:00:00,1,169.399976,400.770039,96.810199,39.292092,model3,18,4 days 21:00:00,3.0
217,2015-01-10 04:00:00,1,162.615800,443.458136,111.995289,44.849501,model3,18,4 days 22:00:00,3.0
218,2015-01-10 05:00:00,1,188.415757,416.819764,105.890248,41.575783,model3,18,4 days 23:00:00,3.0
219,2015-01-10 06:00:00,1,167.756697,448.318668,110.581024,40.992397,model3,18,5 days 00:00:00,3.0


In [21]:

df['error_count_in_last_7_days'] = df.groupby('machineID')['error_count'].ffill()

In [79]:
display(df[['datetime', 'machineID', 'error_count', 'error_last_7d']].head())

,datetime,machineID,error_count,error_last_7d
0,2015-01-01 06:00:00,1,0.0,0.0
1,2015-01-01 07:00:00,1,0.0,0.0
2,2015-01-01 08:00:00,1,0.0,0.0
3,2015-01-01 09:00:00,1,0.0,0.0
4,2015-01-01 10:00:00,1,0.0,0.0


In [80]:
# Example of dropping an unnecessary column if it exists (replace 'error_count_in_last_7_days' with the actual column name if different)
if 'error_count_in_last_7_days' in df.columns:
    df.drop(columns=['error_count_in_last_7_days'], inplace=True)

# Display the columns to confirm
display(df.columns)

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'last_maint_datetime', 'errorID', 'error_count',
       'error_last_7d'],
      dtype='object')

In [27]:

df.iloc[510: 600]

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime,errorID,error_count,error_count_in_last_7_days
510,2015-01-22 06:00:00,1,170.178877,463.217153,108.226270,36.847390,model3,18,2 days 00:00:00,NaN,NaN,3.0
511,2015-01-22 07:00:00,1,167.281171,478.458835,86.291905,46.676575,model3,18,2 days 01:00:00,NaN,NaN,3.0
512,2015-01-22 08:00:00,1,186.604316,425.738317,97.274347,35.699761,model3,18,2 days 02:00:00,NaN,NaN,3.0
513,2015-01-22 09:00:00,1,192.190923,411.051520,97.171660,29.242659,model3,18,2 days 03:00:00,NaN,NaN,3.0
514,2015-01-22 10:00:00,1,152.789008,448.457864,98.709506,33.642445,model3,18,2 days 04:00:00,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
595,2015-01-25 19:00:00,1,155.416374,393.339007,101.852816,43.201760,model3,18,5 days 13:00:00,NaN,NaN,2.0
596,2015-01-25 20:00:00,1,169.872298,480.605956,79.825301,34.975204,model3,18,5 days 14:00:00,NaN,NaN,2.0
597,2015-01-25 21:00:00,1,180.185629,396.821966,90.985634,41.433231,model3,18,5 days 15:00:00,NaN,NaN,2.0
598,2015-01-25 22:00:00,1,178.544471,449.158912,92.278158,45.207357,model3,18,5 days 16:00:00,NaN,NaN,2.0


In [98]:
df.drop(columns=['failure'], inplace=True)

In [99]:
 # Step 1: Prepare the failures data for merging
failures_df['failure'] = 1

# Step 2: Merge the failures data with the main DataFrame
# The 'how="left"' ensures that all hours in the telemetry data are kept
df = pd.merge(df, failures_df[['machineID', 'datetime', 'failure']], on=['machineID', 'datetime'], how='left')

# Step 3: Sort the data in descending order for backward fill
# This is a critical step for bfill() to work correctly
df.sort_values(by=['machineID', 'datetime'], inplace=True)


In [110]:
df[76: 100]

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime,error_last_7d,failure,failure_in_next_7d
76,2015-01-04 10:00:00,1,153.143558,440.162685,94.524894,57.411078,model3,18,NaT,3.0,NaN,1
77,2015-01-04 11:00:00,1,215.656488,458.097746,95.036280,51.647981,model3,18,NaT,3.0,NaN,1
78,2015-01-04 12:00:00,1,173.525320,421.728389,100.617527,50.458297,model3,18,NaT,3.0,NaN,1
79,2015-01-04 13:00:00,1,169.501121,454.460114,91.675576,45.951349,model3,18,NaT,3.0,NaN,1
80,2015-01-04 14:00:00,1,129.016707,479.457721,111.575038,49.398412,model3,18,NaT,3.0,NaN,1
81,2015-01-04 15:00:00,1,168.503141,455.536868,83.689837,43.917862,model3,18,NaT,3.0,NaN,1
82,2015-01-04 16:00:00,1,184.640476,365.213804,87.474009,52.048152,model3,18,NaT,3.0,NaN,1
83,2015-01-04 17:00:00,1,176.208034,517.348533,82.400818,46.950492,model3,18,NaT,3.0,NaN,1
84,2015-01-04 18:00:00,1,183.684832,414.481164,111.684659,56.837165,model3,18,NaT,3.0,NaN,1
85,2015-01-04 19:00:00,1,199.755983,425.832560,107.335080,41.674887,model3,18,NaT,3.0,NaN,1


In [108]:
# Example DataFrame: 'df' with 'machineID', 'datetime', 'failure'
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['machineID', 'datetime'])

# Create a temporary column where only failure timestamps are marked
df['failure_marker'] = df.groupby('machineID')['datetime'].transform(lambda x: x.where(df['failure'] == 1))

# Backward fill the failure timestamps within each machine group
df['failure_time'] = df.groupby('machineID')['failure_marker'].bfill()

# Calculate how far each row is from the next failure
df['days_to_failure'] = (df['failure_time'] - df['datetime']).dt.days

# Mark as 1 if failure is within the next 7 days (including today)
df['failure_in_next_7d'] = ((df['days_to_failure'] >= 0) & (df['days_to_failure'] <= 7)).astype(int)

# Optionally, drop helper columns if not needed
df = df.drop(columns=['failure_marker', 'failure_time', 'days_to_failure'])

In [131]:
# Example of dropping unnecessary columns if they exist
if 'error_count_in_last_7_days' in df.columns:
    df.drop(columns=['error_count_in_last_7_days'], inplace=True)

if 'failure' in df.columns:
    df.drop(columns=['failure'], inplace=True)

# Display the columns to confirm
display(df.columns)

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'last_maint_datetime', 'error_last_7d',
       'failure_in_next_7d'],
      dtype='object')

In [113]:
split_point = int(0.8 * len(df[df['machineID'] == 1]))
date = df[df['machineID'] == 1].iloc[split_point]['datetime']

In [114]:
train_df = df[df['datetime'] < date]
test_df = df[df['datetime'] >= date]

In [116]:
df

,datetime,machineID,volt,rotate,pressure,vibration,model,age,last_maint_datetime,error_last_7d,failure_in_next_7d
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,NaT,0.0,1
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,NaT,0.0,1
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,NaT,0.0,1
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,NaT,0.0,1
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,NaT,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
879722,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5,7 days 20:00:00,0.0,0
879723,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5,7 days 21:00:00,0.0,0
879724,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5,7 days 22:00:00,0.0,0
879725,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5,7 days 23:00:00,0.0,0


In [115]:
df.columns

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'last_maint_datetime', 'error_last_7d',
       'failure_in_next_7d'],
      dtype='object')

In [121]:
features = [col for col in df.columns if col not in ['datetime', 'machineID', 'failure_in_next_7d', 'model']]
target = 'failure_in_next_7d'

In [122]:
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

In [124]:
X_train

,volt,rotate,pressure,vibration,age,last_maint_datetime,error_last_7d
0,176.217853,418.504078,113.077935,45.087686,18,NaT,0.0
1,162.879223,402.747490,95.460525,43.413973,18,NaT,0.0
2,170.989902,527.349825,75.237905,34.178847,18,NaT,0.0
3,162.462833,346.149335,109.248561,41.122144,18,NaT,0.0
4,157.610021,435.376873,111.886648,25.990511,18,NaT,0.0
...,...,...,...,...,...,...,...
877966,175.534349,398.151207,85.798397,37.165525,5,9 days 17:00:00,2.0
877967,166.333239,453.202314,82.475879,40.038595,5,9 days 18:00:00,2.0
877968,182.764144,380.494932,101.311429,41.910656,5,9 days 19:00:00,2.0
877969,173.807904,554.676945,102.280023,33.937532,5,9 days 20:00:00,2.0


In [133]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

# Convert 'last_maint_datetime' from timedelta to numerical (number of days)
X_train.loc[:, 'last_maint_datetime'] = X_train['last_maint_datetime']
X_test.loc[:, 'last_maint_datetime'] = X_test['last_maint_datetime']

# Handle potential NaNs created by the conversion (e.g., fill with a placeholder value or the mean/median)
# For simplicity, we'll fill with a large number assuming a large timedelta means very long since last maint
X_train.loc[:, 'last_maint_datetime'] = X_train['last_maint_datetime'].fillna(X_train['last_maint_datetime'].max() + 1)
X_test.loc[:, 'last_maint_datetime'] = X_test['last_maint_datetime'].fillna(X_test['last_maint_datetime'].max() + 1)


# --- Assuming X_train, y_train, X_test, y_test are already defined ---

# 1. Instantiate the Logistic Regression model with class_weight='balanced'
model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)

# 2. Train the model on the training data
model.fit(X_train, y_train)

# 3. Make predictions on the test data
# We'll use predict_proba to get probabilities, which is better for this problem
y_pred_proba = model.predict_proba(X_test)

# The model outputs probabilities for both classes. We want the probability of failure (class 1)
y_pred_proba_positive = y_pred_proba[:, 1]

# Now, we need to convert probabilities to a binary prediction (0 or 1).
# We'll use a threshold of 0.5 for now, but we can change this later.
y_pred = (y_pred_proba_positive > 0.5).astype(int)

In [134]:
# 4. Evaluate the model
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.2486
Recall: 0.5768
F1-score: 0.3474


In [136]:
import xgboost as xgb

In [137]:
failure_count = y_train.sum()
no_failure_count = len(y_train) - failure_count
scale_pos_weight = no_failure_count / failure_count

# Instantiate the XGBoost classifier with the class imbalance weight
model_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)


In [138]:
model_xgb.fit(X_train, y_train)

# Make predictions on the test data
y_pred_xgb = model_xgb.predict(X_test)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:05:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [139]:
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

print(f"XGBoost Model Performance on Test Data:")
print(f"Precision: {precision_xgb:.4f}")
print(f"Recall:    {recall_xgb:.4f}")
print(f"F1-Score:  {f1_xgb:.4f}")

XGBoost Model Performance on Test Data:
Precision: 0.2898
Recall:    0.9539
F1-Score:  0.4446


In [140]:
# Assuming model_xgb is your trained XGBoost model
y_pred_proba = model_xgb.predict_proba(X_test)

# Get the probability of the positive class (failure)
y_pred_proba_positive = y_pred_proba[:, 1]

In [153]:
# Set your custom threshold
threshold = 0.7

# Convert the probabilities to a binary prediction based on the new threshold
y_pred_custom = (y_pred_proba_positive > threshold).astype(int)

In [154]:

from sklearn.metrics import precision_score, recall_score, f1_score

precision_custom = precision_score(y_test, y_pred_custom)
recall_custom = recall_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom)

print(f"Model Performance with a threshold of {threshold}:")
print(f"Precision: {precision_custom:.4f}")
print(f"Recall:    {recall_custom:.4f}")
print(f"F1-Score:  {f1_custom:.4f}")

Model Performance with a threshold of 0.7:
Precision: 0.3619
Recall:    0.6339
F1-Score:  0.4607
